<a href="https://colab.research.google.com/github/arslan-r/republican_or_democrat/blob/master/Copy_of_hw1_naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Naive Bayes

Naive Bayes will be used for this Machine Learning project. Explanation was obtained from the following website.
 http://theprofessionalspoint.blogspot.com/2019/03/advantages-and-disadvantages-of-naive.html

Bayes theorem is represented with the following expression:

P(A|B) = P(B|A) * P(A) / P(B)

"Here, we calculate the probability of event A, given that event B has occurred.
The right hand side of the equation consists of the probability of event B, given that event A has occurred, multiplied by the ratio of probability of event A to Probability of event B."

#ADVANTAGES
-- Given that data points are independent from each other, this model has a decent performance.

-- Minimum training data is required

-- Easy to implement (bit of a head scratcher as I'm typing this though.  It's ok I got this. Lets take is step by step.)

#DISADVANTAGES
-- Data is assumed to be independent. It almost never is. A lot of variables depend on one another

-- If we dont train the model right, and it encounters a new variable in the test data, it would be impossible to predict data.




Data obtained from https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records.




In [2]:
import pandas 
import urllib.request  
import numpy   # Numerical python.
from decimal import *

# Download the data.
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data"
response = urllib.request.urlopen(url)

# Interpret text data into pandas data frame and simplify. Interpret 'abstain' votes as 'no'.
dataset  = pandas.read_csv(response, header=None, true_values=['y'], false_values=['n','?'])

# Set the column names.
names = ['label'] + [f'vote_{i}' for i in range(16)]
dataset.columns = names

# Tells pandas that this is a categorical feature.
dataset['label'] = pandas.Categorical(dataset['label'])

print("Dataset shape: ", dataset.shape)
dataset.head() # Prints first 5 examples from the data set.

Dataset shape:  (435, 17)


,label,vote_0,vote_1,vote_2,vote_3,vote_4,vote_5,vote_6,vote_7,vote_8,vote_9,vote_10,vote_11,vote_12,vote_13,vote_14,vote_15
0,republican,False,True,False,True,True,True,False,False,False,True,False,True,True,True,False,True
1,republican,False,True,False,True,True,True,False,False,False,False,False,True,True,True,False,False
2,democrat,False,True,True,False,True,True,False,False,False,False,True,False,True,True,False,False
3,democrat,False,True,True,False,False,True,False,False,False,False,True,False,True,False,False,True
4,democrat,True,True,True,False,True,True,False,False,False,False,True,False,True,True,True,True


In [10]:


X = numpy.array(dataset.iloc[:,1:]) # Convert input features into Numpy array.
Y = dataset['label'].cat.codes # Converts string labels to binary values.

# Data is split into train and test set. First 335 examples will be used for training.
num_train = 335
Xtrain = X[0:num_train, :]
Xtest  = X[num_train:,:]




# Sum of Reps and sum of Dems
Ytrain = Y[0:num_train]
Ytest = Y[num_train:,]
sumOfReps = Ytrain[Ytrain == 1].size
print('Sum of Republicans is', sumOfReps)
sumOfDems = Ytrain[Ytrain == 0].size
print('Sum of Democrats is', sumOfDems)




# This is a 2x1 Array featuring p(y == 1) and p(y == 0)
ClassProbabilities = numpy.array([(sumOfReps),(sumOfDems)]).reshape(2,1)
ClassProbabilities = numpy.true_divide(ClassProbabilities, Ytrain.size)
#print('Class Probabilities array looks like', ClassProbabilities)
#print('Shape of ClassProbabilities is', ClassProbabilities.shape)




#On/Off switch for printing a full size array OR just the first few examples. Comment out which one you do not want.
numpy.set_printoptions(edgeitems=3,infstr='inf', linewidth=75, nanstr='nan', precision=8, suppress=False, threshold=1000, formatter=None)
#numpy.set_printoptions(threshold=sys.maxsize)




onlyReps = Xtrain[Ytrain==1] #Now we have only the array with republicans. Time to sum it up.
onlyDems = Xtrain[Ytrain==0] #This is only dems




# Finding the probability that a Republican voted 1 to a measure.
YesRepVotesOnVote = numpy.sum(onlyReps, axis=0) 
ProbWhenYis1 = numpy.true_divide(YesRepVotesOnVote, sumOfReps)

# Same for Democrats
YesDemVotesOnVote = numpy.sum(onlyDems, axis=0) 
ProbWhenYis0 = numpy.true_divide(YesDemVotesOnVote, sumOfDems)
#print('YesDemVotesOnVote divided by total reps. Should be decimal.', ProbWhenYis0)
#print('\n')



print('\n')
# Organizing data
TwoByDArray = numpy.array([(ProbWhenYis1), (ProbWhenYis0)])
print('This is 2x16 Array\n', TwoByDArray)
print('\n')
print('2x16 Array shape is', TwoByDArray.shape)




# Each cell will have the probability of each republican that voted on each vote.
TwoxDArray = numpy.array([(sumOfReps),(sumOfDems)]).reshape(2,1)






Sum of Republicans is 127
Sum of Democrats is 208


This is 2x16 Array
 [[0.16535433 0.46456693 0.11023622 0.96850394 0.94488189 0.86614173
  0.23622047 0.12598425 0.09448819 0.54330709 0.08661417 0.81889764
  0.81102362 0.94488189 0.08661417 0.54330709]
 [0.58653846 0.40384615 0.88461538 0.03365385 0.16346154 0.41346154
  0.77884615 0.83653846 0.74519231 0.45192308 0.44230769 0.11538462
  0.24519231 0.33173077 0.62980769 0.64903846]]


2x16 Array shape is (2, 16)


In [0]:
def generative_model(Xtrain, Ytrain):
    ''' 
    Implements a generative algorithm on binary data.
    Inputs
        Xtrain: NxD matrix of features.
        Ytrain: N vector of class labels

    Returns
        p_label: Length 2 vector of class probabilities.
        p_votes: 2xD Matrix where entry i,j is p(x_j|v=i).
    ''' 
    
    Ytrain = Y[0:num_train]
    Ytest = Y[num_train:,]
    sumOfReps = Ytrain[Ytrain == 1].size
    print('Sum of Republicans is', sumOfReps)
    sumOfDems = Ytrain[Ytrain == 0].size
    print('Sum of Democrats is', sumOfDems)

    p_label = numpy.array([(sumOfReps),(sumOfDems)]).reshape(2,1)
    p_label = numpy.true_divide(p_label, Ytrain.size) #should be ok
    



    onlyReps = Xtrain[Ytrain==1] # Now we have only the array with republicans. Time to sum it up.
    onlyDems = Xtrain[Ytrain==0] # This is only dems




    YesRepVotesOnMeasure = numpy.sum(onlyReps, axis=0) # a 1 x16 array for Reps where each index has the sum of all columns
    ProbWhenYis1 = numpy.true_divide(YesRepVotesOnMeasure, sumOfReps) #Probability

    YesDemVotesOnMeasure = numpy.sum(onlyDems, axis=0) # a 1 x16 array for Dems where each index has the sum of all columns
    ProbWhenYis0 = numpy.true_divide(YesDemVotesOnMeasure, sumOfDems) #Probability

    p_votes = numpy.array([(ProbWhenYis1), (ProbWhenYis0)]) # 2x16 Array that has probability of each vote given the person is rep/dem



    
    return p_label, p_votes
    

 
  
def discriminative_model(p_label, p_votes, Xtest):
    '''
    Implements Naive Bayes Classification.
    Inputs
      p_label, p_votes: From generative_model.
      Xtest: 2x16 matrix of binary features.
    
    Outputs 
      pred_prob: Probability of an individual being a Republican (1)
    '''


    # For each individual, take the probability that they will vote on a certain measure. If they vote 1, use the probability that they will vote 1. 
    # if they vote 0, use (1 - prob they will vote 1)
    # Multiply this through to get the probability that they are republican
    # Do the same for Democrat.

    multByPLabelArrRep = []
    for row in Xtest: #go through each row, multiplying by p_votes.
      testVar1 = 1
      counter = 0
      multiplier = 1
      for var in row:
        if var == 1:
            testVar1 = testVar1*p_votes[0, counter]
            counter+=1
        else:
            testVar1 = testVar1*(1 - p_votes[0, counter])
            counter+=1

      multByPLabelArrRep.append(testVar1)


    # Turn the list into an array. Flatten in.
    multByPLabelArrRep = numpy.asarray(multByPLabelArrRep)
    multByPLabelArrRep = multByPLabelArrRep.flatten()




    # Do the same as above for Democrats.
    multByPLabelArrDem = []
    for row in Xtest: 
      testVar1 = 1
      counter = 0
      multiplier = 1
      for var in row:
        if var == 1:
            testVar1 = testVar1*p_votes[1, counter] 
            counter+=1
            
        else:
            testVar1 = testVar1*(1 - p_votes[1, counter])
            counter+=1
        
      multByPLabelArrDem.append(testVar1)

    # Turn the list into an array. Flatten in.
    multByPLabelArrDem = numpy.asarray(multByPLabelArrDem)
    multByPLabelArrDem = multByPLabelArrDem.flatten()



    # Fix the naming here. Not important but looks better.
    probOfBeingRep = multByPLabelArrRep
    probOfBeingDem = multByPLabelArrDem



    # Multiply by p(y)
    probOfBeingRep = multByPLabelArrRep * p_label[0,]
    probOfBeingDem = multByPLabelArrDem * p_label[1,]



    # Standardize 
    index = 0
    repOverRepAndDem = []
    demOverRepAndDem = []
    while index < len(probOfBeingRep) :

      repVar = probOfBeingRep[index]/(probOfBeingRep[index]+probOfBeingDem[index])
      demVar = probOfBeingDem[index]/(probOfBeingRep[index]+probOfBeingDem[index])

      repOverRepAndDem.append(repVar)
      demOverRepAndDem.append(demVar)

      index += 1
    
    repOverRepAndDem = numpy.asarray(repOverRepAndDem)
    demOverRepAndDem = numpy.asarray(demOverRepAndDem)




    pred_prob = repOverRepAndDem
    

    # Log probability is useful when the numbers get pretty small. 
    natLog = 0

    for var in pred_prob:
      natLog += numpy.log(var)

    print('\n')
    print('This is the natural log', natLog)
    print('\n')
    print('\n')
    print('Preditions on the test points (probability that an individual is a republican):\n', pred_prob)
    return pred_prob



def accuracy(y_true, y_predicted): 
    ''' 
    Calculates the fraction of correct predictions.
    '''

    # Take the test data set. Compare if each individual is as predicted. Get accuracy.
    y_true = numpy.asarray(y_true)
    y_predicted = numpy.asarray(y_predicted)
    y_true = y_true[335:435]
    yTrueSize = len(y_true)

    index = 0
    totalCorrect = 0
    
    while index<yTrueSize:
      if y_predicted[index] > .5:
        y_predicted[index] = 1
      else:
        y_predicted[index] = 0
      if y_true[index] == y_predicted[index]:
        totalCorrect += 1
      index+=1
    
    print('\n')
    percentCorrect = (totalCorrect/(len(y_true)))*100
    print('Percent accuracy:', percentCorrect)
    print('\n')

    pass




def logProb(stuff):

    # For some reason numbers are rounded when passed into other functions? Need to look into this

    natLog = 0

    print('\n')
    print('This is the data passed in', stuff)
    
    for var in stuff:
      #print('This is var', var)
      #print('This is the natural log', var)
      natLog += numpy.log(var)

    print('\n')
    print('This is the natural log', natLog)
    print('\n')

    pass




    

In [16]:
p_label, p_votes = generative_model(Xtrain, Ytrain)
print('Label priors:', p_label)
print('\n')
print('Conditional vote probabilities:\n', p_votes)
print('\n')

predictionOnTest = discriminative_model(p_label, p_votes, Xtest)

print(accuracy(Y, predictionOnTest ))
print('\n')

tempVar = predictionOnTest.astype(float)






Sum of Republicans is 127
Sum of Democrats is 208
Label priors: [[0.37910448]
 [0.62089552]]


Conditional vote probabilities:
 [[0.16535433 0.46456693 0.11023622 0.96850394 0.94488189 0.86614173
  0.23622047 0.12598425 0.09448819 0.54330709 0.08661417 0.81889764
  0.81102362 0.94488189 0.08661417 0.54330709]
 [0.58653846 0.40384615 0.88461538 0.03365385 0.16346154 0.41346154
  0.77884615 0.83653846 0.74519231 0.45192308 0.44230769 0.11538462
  0.24519231 0.33173077 0.62980769 0.64903846]]




This is the natural log -799.5376351844181




Preditions on the test points (probability that an individual is a republican):
 [9.99999899e-01 4.95640927e-06 3.63198586e-11 1.80963571e-09
 9.99998953e-01 9.99997918e-01 4.11813135e-10 6.62684047e-05
 9.75348251e-01 4.48636963e-11 9.99949158e-01 9.99999981e-01
 9.99999913e-01 2.82386635e-09 9.85339654e-01 8.03188656e-07
 8.08427700e-01 4.10067045e-04 9.99888994e-01 4.41144464e-09
 1.81942458e-05 9.99999983e-01 9.99999983e-01 3.24919501e-05
 9.9999